# 法律助理

测试和实验GPT3.5在中国法律助理场景的应用：对中国法律的理解，对法律条款和陈述的综合分析能力的测试。
尝试利用以前bar考的题库建立一个测试集，在其基础上对基础模型的能力建立比较准确的认知。后面可以体系性的开发提效。


In [ ]:
#!pip install datasets

构建框架：数据

In [ ]:
from datasets import load_dataset

#dataset = load_dataset("truthful_qa", "generation")
#UTF8 Comma delimited CSV file exported from exel
dataset = load_dataset("csv", data_files="/mnt/c/Users/qiang/Downloads/BarExamData_CSV.csv")

In [ ]:
examples = list(dataset['train'])


In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chains import LLMChain
from langchain.llms import OpenAI
GPT4='GPT-4-32k'
GPT3_5='text-davinci-003'
llm = OpenAI(model_name=GPT3_5, temperature=0.0, max_tokens=1500)

最简单的推理测试：找到好的prompt很不容易，需要对法律场景、问题类型、模型与人在这样问题上的思维方式及其偏见（inductive bias）有深刻的理解后（经过很多尝试后）才能生成。

In [ ]:
qTemplate="请仔细理解以下法律条款和陈述后回答问题 法律条款：{law} 陈述：{prefix} {statement} 问题：{question}"

question = "以上法律条款适用于以上陈述的先决条件是什么？这个条件满足了吗？为什么？陈述是正确的吗？为什么？"

qPrompt = PromptTemplate(input_variables=["law","prefix", "statement", "question"], template=qTemplate)
qChain = LLMChain(llm=llm, verbose=True, prompt=qPrompt)

Add the question prompt to each data entry which can be modified independent of input data.
Evaluating all the data at the same time.

In [ ]:
def generator_add(list_):
    yield from ({'question':question, 'law':entry['law'], 'prefix':entry['prefix'], 'statement':entry['statement'], 'answer':entry['answer']} for entry in list_)

# Prints all results in once
eles = list(generator_add(examples))

predictions = qChain.apply(eles)


In [ ]:
predictions

把上面复杂、细节的回答转换为简单的答案（正确或错误）以便最后评估结果。

In [ ]:
aTemplate="{text} 问题：陈述是正确的还是错误的（请回答正确或错误）？"

aPrompt = PromptTemplate(input_variables=["text"], template=aTemplate)
aChain = LLMChain(llm=llm, verbose=True, prompt=aPrompt)

predictionsAns = aChain.apply(predictions)


In [ ]:
predictionsAns

Manual Evaluate/Probe one at a time.

In [ ]:

#i=1
#qChain.run(law=eles[i]['law'], prefix=eles[i]['prefix'], statement=eles[i]['statement'], question=eles[i]['question'])


## Evaluate performance
Now we can evaluate the predictions. The first thing we can do is look at them by eye.

In [ ]:
from langchain.evaluation.qa import QAEvalChain

Next, we can use a language model to score them programatically

In [ ]:
llm = OpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(eles, predictionsAns, question_key="statement", prediction_key="text", answer_key='answer')

In [ ]:
graded_outputs

We can add in the graded output to the `predictions` dict and then get a count of the grades.

In [ ]:
for i, prediction in enumerate(predictionsAns):
    prediction['grade'] = graded_outputs[i]['text']
    prediction['answer'] = eles[i]['answer']

In [ ]:
from collections import Counter
Counter([pred['grade'] for pred in predictionsAns])

We can also filter the datapoints to the incorrect examples and look at them.

In [ ]:
incorrect = [pred for pred in predictionsAns if pred['grade'] == " INCORRECT"]
correct = [pred for pred in predictionsAns if pred['grade'] == " CORRECT"]

In [ ]:
correct